In [ ]:
from sqlalchemy import create_engine
from pandas.io.sql import read_sql

conn_string = 'postgresql://iangow.me/crsp'
engine = create_engine(conn_string)

In [ ]:
file_name = '999419_T'

sql = """
    SELECT speaker_name, role, context,
        file_name, last_update, speaker_number, speaker_text
    FROM streetevents.speaker_data
    WHERE file_name='%s'
    ORDER BY context, speaker_number
""" % (file_name)

df = read_sql(sql, engine)
some_text = df["speaker_text"][2]
some_text

In [ ]:
def numbers(text):
 
    import re
     
    # Regular expression to pick up months as described in Lundholm, Rogo and Zhang (2014)
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
                    'August', 'September', 'October', 
                    'November', 'December']     
    months_abbrev = [ month[0:3] + '\.?' for month in months]
    month_re = '(?:' + '|'.join(months + months_abbrev) + ')'
 
    # Years are four-digit numbers without commas.
    # I have a better rule using the fact that years will begin with 19 or 20, etc.
    years = '\s*\d{4}\.?'
 
    # Keep matching numbers, as well as month portion, if any
    matches =  re.findall('(?:' + month_re + '\s+)?[0-9][0-9,\.]*', text)
 
    # return matches that aren't years and aren't preceded by months
    return [ a_match for a_match in matches
                if not re.match(years, a_match) 
                    and not re.match(month_re, a_match)]

In [ ]:
numbers(some_text)

In [ ]:
len(numbers(some_text))

In [ ]:
len(some_text)

In [ ]:
def numerical_intensity(text):
    return len(numbers(text))/len(text)

In [ ]:
df['num_int'] = df['speaker_text'].map(numerical_intensity)

In [ ]:
df

In [ ]:
# Push data to PostgreSQL database
df.to_sql('num_intensity', engine,
         if_exists="replace", index=False)